In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [ ]:
try:
    path = "../tiles"
    os.mkdir(path)
    classes = ['gl3_yes', 'gl3_no', 'gl4_yes', 'gl4_no', 'gl5_yes', 'gl5_no']
    t_types = ['training', 'testing']
    for item in t_types:
        path1 = os.path.join(path, item)
        os.mkdir(path1)
        for item in classes:
            path2 = os.path.join(path1, item)
            os.mkdir(path2)
except OSError as error:
    print(error)

In [ ]:

for item in classes[0:2]:
    os.mkdir('../'+item)
    with zipfile.ZipFile('../input/sorting-tiles-into-classes'+item[:3]+'/'+item+'.zip') as z:
        z.extractall('../'+item)

In [ ]:
for item in classes[0:2]:
    print(len(os.listdir('../'+item)))

In [ ]:
sz = min([len(os.listdir('../'+item)) for item in classes[0:2]])
files = os.listdir('../'+classes[1])
shuffl = random.sample(files, len(files))
discard = shuffl[sz:]
for item in discard:
    os.remove('../'+classes[1]+'/'+item)
print(len(os.listdir('../'+classes[1])))

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    source_files = os.listdir(SOURCE)
    good_files = [f for f in source_files if os.path.getsize(os.path.join(SOURCE, f)) != 0]
    shuffle = random.sample(good_files, len(good_files))
    stop = int(len(good_files) * SPLIT_SIZE)
    train_files = shuffle[:stop]
    test_files = shuffle[stop:]
    for item in train_files:
        f_source  = os.path.join(SOURCE, item)
        f_dest = os.path.join(TRAINING, item)
        copyfile(f_source, f_dest)
    for item in test_files:
        f_source  = os.path.join(SOURCE, item)
        f_dest = os.path.join(TESTING, item)
        copyfile(f_source, f_dest)

In [ ]:
yes3_source_dir = '../gl3_yes/'
training_yes3_dir = '../tiles/training/gl3_yes/'
testing_yes3_dir = '../tiles/testing/gl3_yes/'

no3_source_dir = '../gl3_no/'
training_no3_dir = '../tiles/training/gl3_no/'
testing_no3_dir = '../tiles/testing/gl3_no/'

yes4_source_dir = '../gl4_yes/'
training_yes4_dir = '../tiles/training/gl4_yes/'
testing_yes4_dir = '../tiles/testing/gl4_yes/'

no4_source_dir = '../gl4_no/'
training_no4_dir = '../tiles/training/gl4_no/'
testing_no4_dir = '../tiles/testing/gl4_no/'

yes5_source_dir = '../gl5_yes/'
training_yes5_dir = '../tiles/training/gl5_yes/'
testing_yes5_dir = '../tiles/testing/gl5_yes/'

no5_source_dir = '../gl5_no/'
training_no5_dir = '../tiles/training/gl5_no/'
testing_no5_dir = '../tiles/testing/gl5_no/'

split_size = .8
split_data(yes3_source_dir, training_yes3_dir, testing_yes3_dir, split_size)
split_data(no3_source_dir, training_no3_dir, testing_no3_dir, split_size)

#split_data(yes4_source_dir, training_yes4_dir, testing_yes4_dir, split_size)
#split_data(no4_source_dir, training_no4_dir, testing_no4_dir, split_size)

#split_data(yes5_source_dir, training_yes5_dir, testing_yes5_dir, split_size)
#split_data(no5_source_dir, training_no5_dir, testing_no5_dir, split_size)

In [ ]:
print(len(os.listdir('../tiles/training/gl3_yes/')))
print(len(os.listdir('../tiles/training/gl3_no/')))
print(len(os.listdir('../tiles/testing/gl3_yes/')))
print(len(os.listdir('../tiles/testing/gl3_no/')))

#print(len(os.listdir('../tiles/training/gl4_yes/')))
#print(len(os.listdir('../tiles/training/gl4_no/')))
#print(len(os.listdir('../tiles/testing/gl4_yes/')))
#print(len(os.listdir('../tiles/testing/gl4_no/')))

#print(len(os.listdir('../tiles/training/gl4_yes/')))
#print(len(os.listdir('../tiles/training/gl4_no/')))
#print(len(os.listdir('../tiles/testing/gl4_yes/')))
#print(len(os.listdir('../tiles/testing/gl4_no/')))

In [ ]:
for item in t_types:
    path = "../tiles"
    path1 = os.path.join(path, item)
    for item in classes:
        path2 = os.path.join(path1, item)
        if os.path.isdir(path2):
            if len(os.listdir(path2)) == 0:
                shutil.rmtree(path2)
        else:
            print(path2+ ' doesnae exist')

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

import cv2
from tensorflow.keras.applications import ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [ ]:
pre_trained_model = ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3))

In [ ]:
pre_trained_model.summary()

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:

from tensorflow.keras.optimizers import RMSprop
x = tf.keras.layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0005), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [ ]:
train_dir = '../tiles/training'
validation_dir = '../tiles/testing'
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 50,
                                                    class_mode = 'binary', 
                                                    target_size = (224, 224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 50,
                                                          class_mode  = 'binary', 
                                                          target_size = (224, 224))

In [ ]:
from tensorflow.python.keras.callbacks import ModelCheckpoint
cb_checkpointer = ModelCheckpoint(filepath = '../working/best.hdf5', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 2446,
            epochs = 30,
            validation_steps = 661,
            verbose = 2,
            callbacks=[cb_checkpointer])

In [ ]:
model.save('gl3_model')

In [ ]:
dirs = ['../gl3_no', '../gl3_yes', '../tiles']
for item in dirs:
    shutil.rmtree(item)